In [39]:
# Parameters
artificial_humans = "../../data/training/ah_10/data/model.pt"
artificial_humans_model = "graph"
output_path = "../../data/manager/simulate_rule/v1/"
n_episode_steps = 16
s = 0
b = 0
c = 0
n_episodes = 2


In [40]:
%load_ext autoreload
%autoreload 2

import os
import torch as th
from itertools import count
from aimanager.utils.utils import make_dir

from aimanager.manager.memory import Memory
from aimanager.manager.environment import ArtificialHumanEnv
from aimanager.artificial_humans import AH_MODELS
from aimanager.utils.array_to_df import using_multiindex

output_path = os.path.join(output_path, 'data')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
class RuleManager:
    def __init__(self, s, b, c):
        self.s = s
        self.b = b
        self.c = c

    def get_punishment(self, contributions, **_):
        punishments = (20-contributions) * self.s + (20 != contributions).to(th.float) * self.c - self.b
        punishments = punishments.round().to(th.int64)
        punishments = th.minimum(th.maximum(punishments, th.zeros_like(punishments)), th.full_like(punishments, 30))
        return punishments

In [42]:
device = th.device('cpu')
rec_device = th.device('cpu')
artifical_humans = AH_MODELS[artificial_humans_model].load(artificial_humans).to(device)

env = ArtificialHumanEnv(
    artifical_humans=artifical_humans, n_agents=4, n_contributions=21, n_punishments=31, episode_steps=n_episode_steps, device=device)

recorder = Memory(n_episodes=n_episodes, n_episode_steps=n_episode_steps, output_file=None, device=device)

for episode in range(n_episodes):
    manager = RuleManager(s=s, b=b, c=c)
    state = env.init_episode()
    for step in count():
        action = manager.get_punishment(**state)
        state = env.punish(action)
        recorder.add(**state, episode_step=step)
        state, reward, done = env.step()
        if done:
            break
    recorder.next_episode(episode)

In [43]:
punishments = using_multiindex(recorder.memory['punishments'].numpy(), columns=['idx','round_number', 'agent'], value_name='punishments')
common_good = using_multiindex(recorder.memory['common_good'].numpy(), columns=['idx','round_number', 'agent'], value_name='common_good')
contributions = using_multiindex(recorder.memory['contributions'].numpy(), columns=['idx','round_number', 'agent'], value_name='contributions')

df = punishments.merge(common_good).merge(contributions)

df = df.drop(columns=['idx', 'agent'])

df = df.groupby(['round_number']).mean().reset_index()
df['s'] = s
df['b'] = b
df['c'] = c


make_dir(output_path)
df.to_csv(os.path.join(output_path, 'trace.csv'))